In [2]:
import numpy as np
import pandas as pd
import os

import csv
import yaml

from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import Namespace

from tqdm import tqdm

# Import Data

## Import DRKG (with rdflib.Graph) -- outdated

In [2]:
drkg_path = "data/drkg/drkg.tsv"
ns = Namespace("http://drkg.org/graph/")

In [3]:
drkg = Graph()

print("Loading DRKG database from tsv...")
with open(drkg_path, 'r') as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    for row in reader:
        if len(row) == 3:
            subject = ns[row[0].replace(' ', '_').replace('|', '_')]
            predicate = ns[row[1].replace(' ', '_').replace('>', '_')]
            # Determine if object is a URI or a Literal
            object = URIRef(row[2].replace(' ', '_').replace('|', '_')) if row[2].startswith('http') else Literal(row[2].replace(' ', '_').replace('|', '_'))
            drkg.add((subject, predicate, object))
            #print(f'Added:\n {subject}, \n {predicate}, \n {object}.')

Loading DRKG database from tsv...


In [46]:
cnt = 0

def raw2str(row):
    # remove uri and clean predicate
    s,p,o = row
    if "drkg" in s:
        return str(s).split("/")[-1], clean_pred(str(p).split("/")[-1]), str(o).split("/")[-1]
    else:
        return str(s).split("/")[-1], str(p).split("/")[-1], str(o).split("/")[-1]

def clean_pred(p):
    return str(p).split("::")[1]    # 0 - source, 1 - predicate, 2 - head & tail entity types

In [ ]:
# Test
for row in drkg:
    s,p,o = raw2str(row)

    if "MESH" in s or "MESH" in o:
        print(s,p,o)
        cnt += 1
    if cnt > 50: break

## Import DRKG (with pandas)
Source: https://github.com/gnn4dr/DRKG/blob/master/raw_graph_analysis/Jaccard_scores_among_all_edge_types_in_DRKG.ipynb

In [ ]:
drkg_file = 'data/drkg/drkg.tsv'
drkg_df = pd.read_csv(drkg_file, sep="\t", header=None)     # header fix to original code
drkg_triplets = drkg_df.values.tolist()

In [94]:
drkg_df.tail()

,0,1,2
5874256,Gene::29099,STRING::REACTION::Gene:Gene,Gene::1643
5874257,Gene::51645,STRING::REACTION::Gene:Gene,Gene::3183
5874258,Gene::865,STRING::CATALYSIS::Gene:Gene,Gene::983
5874259,Gene::1066,STRING::BINDING::Gene:Gene,Gene::7365
5874260,Gene::6118,STRING::BINDING::Gene:Gene,Gene::1111


### Entity dictionary
Dictionary format:  {**keys**: Entity Type (str), **values**: Dict{}}

Sub-dictionary format: {**keys**: Entity Name (str), **values**: Entity Index (int)}

Example: *Gene {'Gene::2157': 0, 'Gene::5264': 1, 'Gene::2158': 2, 'Gene::3309': 3, 'Gene::28912': 4, ...}*

In [9]:
drkg_entity_dictionary={}
def insert_entry(entry,ent_type,dic):
    if ent_type not in dic:
        dic[ent_type]={}
    ent_n_id=len(dic[ent_type])
    if entry not in dic[ent_type]:
         dic[ent_type][entry]=ent_n_id
    return dic

for triple in drkg_triplets:
    src = triple[0]
    split_src=src.split('::')
    src_type=split_src[0]
    dest = triple[2]
    split_dest=dest.split('::')
    dest_type=split_dest[0]
    insert_entry(src,src_type,drkg_entity_dictionary)
    insert_entry(dest,dest_type,drkg_entity_dictionary)

In [127]:
# DEBUG entity dict
for k,v in drkg_entity_dictionary.items():
    # print(k,v)
    string = "{"+k+" {"
    for a,tuple in enumerate(v.items()):
        ent, eid = tuple
        string = string + f"{ent}-{eid}"
        if a == 3: break
        string += ", "
    print(string + ", ... }}")
        

{Gene {Gene::2157-0, Gene::5264-1, Gene::2158-2, Gene::3309-3, ... }}
{Compound {Compound::DB02573-0, Compound::DB05105-1, Compound::DB00244-2, Compound::DB00684-3, ... }}
{Disease {Disease::SARS-CoV2 E-0, Disease::SARS-CoV2 M-1, Disease::SARS-CoV2 N-2, Disease::SARS-CoV2 Spike-3, ... }}
{Atc {Atc::B01AE02-0, Atc::L01XC06-1, Atc::R05CB13-2, Atc::L01XX29-3, ... }}
{Tax {Tax::10090-0, Tax::10116-1, Tax::9913-2, Tax::4932-3, ... }}
{Biological Process {Biological Process::GO:0071357-0, Biological Process::GO:0098780-1, Biological Process::GO:0055088-2, Biological Process::GO:0010243-3, ... }}
{Symptom {Symptom::D007383-0, Symptom::D021501-1, Symptom::D010146-2, Symptom::D013064-3, ... }}
{Anatomy {Anatomy::UBERON:0002110-0, Anatomy::UBERON:0001980-1, Anatomy::UBERON:0001760-2, Anatomy::UBERON:0002360-3, ... }}
{Molecular Function {Molecular Function::GO:0042803-0, Molecular Function::GO:0016274-1, Molecular Function::GO:0015179-2, Molecular Function::GO:0005085-3, ... }}
{Pharmacologic Cl

### Node & Edge dictionary
Edge dictionary format: {*keys*: Predicate Type (str), *values*: List **[** head Entity Index (int), tail Entity Index (int) **]**}

Node dictionary format: {*keys*: Entity Type (str), *values*: Set **(** Entity Index* (int) **)**}  
**head and tail entity index added separately*

In [ ]:
edge_dictionary={}
node_dictionary={}
for triple in drkg_triplets:
    src = triple[0]
    split_src=src.split('::')
    src_type=split_src[0]
    dest = triple[2]
    split_dest=dest.split('::')
    dest_type=split_dest[0]
    
    src_int_id=drkg_entity_dictionary[src_type][src]
    dest_int_id=drkg_entity_dictionary[dest_type][dest]
    
    pair=[(src_int_id,dest_int_id)]
    etype=triple[1]
    if etype in edge_dictionary:
        edge_dictionary[etype]+=pair
    else:
        edge_dictionary[etype]=pair
    if etype in node_dictionary:
        node_dictionary[etype].add(src_int_id)
        node_dictionary[etype].add(dest_int_id)
    else:
        node_dictionary[etype]=set()   
        node_dictionary[etype].add(src_int_id)
        node_dictionary[etype].add(dest_int_id)

print(f"Number of unique edge types: {len(edge_dictionary)}")

Number of unique edge types: 107


In [ ]:
for k,v in node_dictionary.items():
    print(type(k),type(v))
    print(k)
    for a in v:
        print(a)
print(len(edge_dictionary))

## Import DrugMechDB (with rdflib.Graph) --outdated

In [ ]:
dmdb_path = "data/drugmech/drugmechdb.yaml"
ns = Namespace("http://drugmech.org/graph/")

dmdb = Graph()

with open(dmdb_path, 'r') as yaml_file:
    data = yaml.safe_load(yaml_file)
    print('YAML loaded')

    count = [0,0,0]
    
    for entry in data:
        # graph_id = entry['graph']['_id']

        for node in entry['nodes']:
            node_id = ns[node['id']]
            label = Literal(node['label'])
            name = Literal(node['name'])
            dmdb.add((node_id, RDF.type, URIRef(ns + 'Node')))
            dmdb.add((node_id, URIRef(ns + 'label'), label))
            dmdb.add((node_id, URIRef(ns + 'name'), name))
            count[1] += 1
            # print('Node :', node)
            
        for link in entry['links']:
            source = URIRef(ns + link['source'])
            target = URIRef(ns + link['target'])
            predicate = URIRef(ns + link['key'].replace(" ", "_"))
            dmdb.add((source, predicate, target))
            count[2] += 1
            # print('Link :', link)
        count[0] += 1
        # print('Entry :', entry)
    print(count)

YAML loaded
[4846, 33009, 32641]


### Step 1
How much overlap between DMDB and DRKG entities?

In [39]:
dmdb_mesh = set()

for row in dmdb:
    s,p,o = raw2str(row)

    if "MESH" in s:
        dmdb_mesh.add(s)
    elif "MESH" in o:
        dmdb_mesh.add(o)

print("Number of MESH-coded entities in DrugMechDB:", len(dmdb_mesh), "/", len(dmdb))

Number of MESH-coded entities in DrugMechDB: 2539 / 27228


In [ ]:
common_mesh = set()


drkg_mesh_cnt = 0
for row in drkg:
    s,p,o = raw2str(row)

    if "MESH" in s:
        drkg_mesh_cnt += 1
        if s in dmdb_mesh:
            common_mesh.add(s)
    elif "MESH" in o:
        drkg_mesh_cnt += 1
        if o in dmdb_mesh:
            common_mesh.add(o)
print("Number of MESH-coded entities in DRKG:", drkg_mesh_cnt, "/", len(drkg))

Number of MESH-coded entities in DRKG: 239803 / 5874258


In [34]:
for i in common_mesh:
    print(i)

print(len(dmdb_mesh))

14767


### Getting Unique Entities

In [51]:
# Get DRKG's unique entities
drkg_entities = set()

for row in tqdm(drkg):
    s,p,o = raw2str(row)

    for ent in [s,o]:
        if not isinstance(ent, Literal):
            drkg_entities.add(ent)

print(f"Number of stored unique entities: {len(drkg_entities)}")


100%|██████████| 5874258/5874258 [00:39<00:00, 148673.11it/s]

Number of stored unique entities: 97238


In [56]:
# Get DMDB's Unique entities
dmdb_entities = set()

for row in tqdm(dmdb):
    s,p,o = raw2str(row)

    for ent in [s,o]:
        if not isinstance(ent, Literal):
            # print(ent)
            dmdb_entities.add(ent)

print(f"Number of stored unique entities: {len(dmdb_entities)}")

100%|██████████| 27228/27228 [00:00<00:00, 246468.18it/s]

Number of stored unique entities: 10962


In [ ]:
# DEBUG         TODO : Literals in entities?
for idx, i in enumerate(dmdb_entities):
    print(i)
    if idx == 30: break

vitamin C
purine nucleotide biosynthetic process
Wheezing
MESH:D001943
Chloramphenicol
MESH:C536777
Paroxysmal nocturnal hemoglobinuria
Cytokine production
MESH:D003907
Protein Kinase C
Cataract
imipenem
Streptococcal infectious disease
Giant Cell Arteritis
Pneumococcal meningitis
UniProt:P0DJD9
MESH:D056806
MESH:D002177
UniProt:P00488
MESH:D003607
Rosuvastatin
UniProt:Q8ZB62
DB:DB01430
Abnormality of temperature regulation
MESH:D008118
Cytokine receptor common subunit gamma
5-hydroxytryptamine receptor 1D
regulation of synaptic transmission, dopaminergic
Lymphoma, Mantle-Cell
Diazepam
Magnesium


In [ ]:
# Comparing unique entity sets
common_entities = set()

for elem in tqdm(dmdb_entities, desc="Comparison in progress"):
    if elem in drkg_entities:
        common_entities.add(elem)

print("Number of common entities:", len(common_entities), "/", (len(drkg_entities) + len(dmdb_entities)))

cnt = 0

for idx, i in enumerate(common_entities):
    if "MESH" in i: cnt += 1       # hard-coded filter...
print(cnt)

Comparison in progress: 100%|██████████| 10962/10962 [00:00<00:00, 1811867.92it/s]

Number of common entities: 10962 / 119162


In [ ]:
# Inefficient entity comparison
common_entities = set()

for row in tqdm(dmdb, desc="Hour-long comparison in progress..."):
    s,p,o = raw2str(row)

    for row2 in drkg:
        s2,p2,o2 = raw2str(row2)

        if s in s2 or s in o2:
            common_entities.add(s)
        if o in s2 or o in o2:
            common_entities.add(o)

print("Number of common entities:", len(common_entities), "/", (len(drkg) + len(dmdb)))

## Import DrugMechDB (with pandas.Dataframe)

Source: https://github.com/SuLab/DrugMechDB/blob/main/data_analysis/figures_DMDB_manuscript.ipynb

In [35]:
drugmech_path = "data/drugmech/drugmechdb.yaml"

with open(drugmech_path, 'r') as fh:
        ind = yaml.safe_load(fh)

In [41]:
from dmdb_data_tools_analysis import *
from collections import defaultdict

In [39]:
all_metapath_nodes = get_metapath_node(ind)
all_metapath_edges = get_metapath_edges(ind)

In [44]:
basic_stats = defaultdict(list)
all_metaedges = []
all_parings = []
all_targets = []
unique_metaedges = []
first_edge_type = []
all_nodes = []

id_to_name = {}
id_to_label = {}

for i, p in enumerate(ind):
    _id = (p["graph"]["_id"])
    drug_id, dis_id = path_to_tup(p)
    paths = get_all_paths(p)
    G = path_to_G(p)
    
    G = add_metaedges(G)
    G = add_meanode_pairs(G)
    
    basic_stats['idx'].append(i) #index
    basic_stats['id'].append(p['graph']['_id']) #DrugMechDB id
    basic_stats['drug'].append(drug_id) #Drug id
    basic_stats['disease'].append(dis_id)#Disease id
    basic_stats['nodes'].append((G.nodes)) #nodes in metapath
    basic_stats['n_nodes'].append(len(G.nodes)) # number of nodes in metapath
    basic_stats['n_edges'].append(len(G.edges)) #number of edges in metapath
    basic_stats['n_paths'].append(len(all_metapath_nodes[_id])) #number of paths
    basic_stats['metapath'].append(all_metapath_nodes[_id])
    basic_stats['metapath_with_edges'].append(all_metapath_edges[_id])

    
    this_metaedges = [G.edges[e]['metaedge'] for e in G.edges]
    
    all_metaedges += this_metaedges
    unique_metaedges += list(set(this_metaedges))
    
    all_parings += [G.edges[e]['mn_pair'] for e in G.edges]
    all_targets += get_targets(G)
    first_edge_type += get_target_metaedges(G)
    all_nodes += list(G.nodes)
    
    id_to_label = {**id_to_label, **get_id_to_type(G)}
    id_to_name = {**id_to_name, **get_id_to_name(G)}
    
dmdb_df = pd.DataFrame(basic_stats)

In [114]:
dmdb_df.head()

,idx,id,drug,disease,nodes,n_nodes,n_edges,n_paths,metapath,metapath_with_edges
0,0,DB00619_MESH_D015464_1,DB:DB00619,MESH:D015464,"(MESH:D000068877, UniProt:P00519, MESH:D015464)",3,2,1,[Drug - Protein - Disease],[Drug - decreases activity of - Protein - caus...
1,1,DB00619_MESH_D034721_1,DB:DB00619,MESH:D034721,"(MESH:D000068877, UniProt:P10721, UniProt:P162...",5,5,1,[Drug - Protein - BiologicalProcess - Disease],[Drug - decreases activity of - Protein - posi...
2,2,DB00316_MESH_D010146_1,DB:DB00316,MESH:D010146,"(MESH:D000082, UniProt:P23219, UniProt:P35354,...",7,8,1,[Drug - Protein - BiologicalProcess - Chemical...,[Drug - decreases activity of - Protein - posi...
3,3,DB00316_MESH_D005334_1,DB:DB00316,MESH:D005334,"(MESH:D000082, reactome:R-HSA-2162123, UBERON:...",5,4,1,[Drug - Pathway - GrossAnatomicalStructure - B...,[Drug - negatively regulates - Pathway - occur...
4,4,DB00945_MESH_D010146_1,DB:DB00945,MESH:D010146,"(MESH:D001241, UniProt:P23219, UniProt:P35354,...",7,7,1,[Drug - Protein - BiologicalProcess - Disease],[Drug - decreases activity of - Protein - posi...


In [125]:
# P.S Careful with drug ID, DrugBank ID in drug, MESH ID in nodes
# Also 'metapath's are a list with one element 
# And also the metapath is confusing (e.g Protein X and Y tie Drug D to Disease Di, D - X - Di + D - Y - Di, and metapaths only map Drug - Protein - Disease)

def compare_ids(row):
    return row['drug'] == list(row['nodes'])[0]

result = dmdb_df.apply(compare_ids, axis=1)
cnt = 0
for res in result:
    if res:
        cnt += 1
        # print(res)
print(f"Number of ['drug'] column drug IDs matching ['nodes'] column drug ID: {cnt}/{len(result)}\n")

for elem in dmdb_df.loc[0,'metapath_with_edges'][0].split(" - "):   # metapath testing
    print(elem)



Number of ['drug'] column drug IDs matching ['nodes'] column drug ID: 166/4846

Drug
decreases activity of
Protein
causes
Disease


In [ ]:
# TODO : fix ONLY IF needed

dmdb_entity_dictionary={}
def insert_entry(entry,ent_type,dic):
    if ent_type not in dic:
        dic[ent_type]={}
    ent_n_id=len(dic[ent_type])
    if entry not in dic[ent_type]:
         dic[ent_type][entry]=ent_n_id
    return dic

for idx, row in dmdb_df.iterrows():
    for n_id, ent in enumerate(list(row['nodes'])):
        templist = row['metapath'][0].split(" - ")
        what = row['metapath_with_edges'][0].split(" - ")
        print(n_id, len(templist), ent, templist, list(row['nodes']), what)
        ent_type = templist[n_id]
        insert_entry(ent,ent_type,dmdb_entity_dictionary)

0 3 MESH:D000068877 ['Drug', 'Protein', 'Disease'] ['MESH:D000068877', 'UniProt:P00519', 'MESH:D015464'] ['Drug', 'decreases activity of', 'Protein', 'causes', 'Disease']
1 3 UniProt:P00519 ['Drug', 'Protein', 'Disease'] ['MESH:D000068877', 'UniProt:P00519', 'MESH:D015464'] ['Drug', 'decreases activity of', 'Protein', 'causes', 'Disease']
2 3 MESH:D015464 ['Drug', 'Protein', 'Disease'] ['MESH:D000068877', 'UniProt:P00519', 'MESH:D015464'] ['Drug', 'decreases activity of', 'Protein', 'causes', 'Disease']
0 4 MESH:D000068877 ['Drug', 'Protein', 'BiologicalProcess', 'Disease'] ['MESH:D000068877', 'UniProt:P10721', 'UniProt:P16234', 'GO:0008283', 'MESH:D034721'] ['Drug', 'decreases activity of', 'Protein', 'positively regulates', 'BiologicalProcess', 'causes', 'Disease']
1 4 UniProt:P10721 ['Drug', 'Protein', 'BiologicalProcess', 'Disease'] ['MESH:D000068877', 'UniProt:P10721', 'UniProt:P16234', 'GO:0008283', 'MESH:D034721'] ['Drug', 'decreases activity of', 'Protein', 'positively regulate

IndexError: list index out of range

In [122]:
dmdb_entity_set = set()

for idx, row in dmdb_df.iterrows():
    for n_id, ent in enumerate(list(row['nodes'])):
        dmdb_entity_set.add(ent)

print("Number of unique entities in dmdb:", len(dmdb_entity_set))

Number of unique entities in dmdb: 5128


# Analysis

## Step 1: Compare Entities

In [ ]:
cnt = 0
common_dict = {}
for e_type,dict in tqdm(drkg_entity_dictionary.items()):
    for elem,eid in dict.items():
        e_type, e_name = elem.split("::")     # TODO : This is just an integer for some classes
        # print(elem)
        if e_name in dmdb_entity_set:
            # print("Common element found: ", elem)
            cnt += 1
            if e_type in common_dict:
                common_dict[e_type]+=[str(e_name)]
            else:
                common_dict[e_type]=[e_name]

print("Number of hits:", cnt)

total = 0
for v,i in common_dict.items():
    print(f"({len(i)})", v, i)
    total += len(i)
print("DEBUG Total:", total)

100%|██████████| 13/13 [00:00<00:00, 447.65it/s]

Number of hits: 1653
(279) Compound ['MESH:C026098', 'CHEBI:15356', 'CHEBI:15889', 'CHEBI:16134', 'CHEBI:16136', 'CHEBI:16412', 'CHEBI:16991', 'CHEBI:17234', 'CHEBI:17855', 'CHEBI:25029', 'CHEBI:26333', 'CHEBI:28044', 'CHEBI:28300', 'CHEBI:35143', 'CHEBI:35366', 'CHEBI:36500', 'MESH:D000728', 'MESH:D001224', 'MESH:D001647', 'MESH:D002216', 'MESH:D003520', 'MESH:D006497', 'MESH:D010710', 'MESH:D012313', 'MESH:D012964', 'MESH:D013256', 'MESH:D016559', 'MESH:D018698', 'MESH:C000599293', 'MESH:C001652', 'MESH:C005435', 'MESH:C006632', 'MESH:C018279', 'MESH:C019248', 'MESH:C020243', 'MESH:C026116', 'MESH:C026563', 'MESH:C029256', 'MESH:C029620', 'MESH:C030852', 'MESH:C032302', 'MESH:C035133', 'MESH:C036619', 'MESH:C036901', 'MESH:C036944', 'MESH:C037178', 'MESH:C041359', 'MESH:C041877', 'MESH:C042734', 'MESH:C043211', 'MESH:C043877', 'MESH:C045463', 'MESH:C045645', 'MESH:C047246', 'MESH:C047781', 'MESH:C048107', 'MESH:C048833', 'MESH:C055122', 'MESH:C055603', 'MESH:C056493', 'MESH:C059205',

# Step 2: Compare Predicates

In [ ]:
# Manual alignment of relationships